# AEROSP 536 Electric Propulsion: Homework 5
Jason Chen

---

## Problem 1: Hall Thruster Performance
Consider the following metrics for a Hall thruster:
| Parameter                      | Symbol       | Value       |
|--------------------------------|--------------|-------------|
| Discharge voltage              | $V_d$        | 300 V       |
| Discharge current              | $I_d$        | 20 A        |
| Mass flow rate                 | $\dot{m}$    | 21.0402 mg/s|
| Cathode voltage                | $V_c$        | 20 V        |
| Magnet power                   | $P_{mag}$    | 31.2 W      |
| Beam current                   | $I_b$        | 15 A        |
| Divergence half-angle          | $\theta$     | 15°         |
| Charge utilization efficiency  | $\eta_q$     | 1.0         |

### Part (a)
Calculate the efficiency of the thruster, assuming charge utilization efficiency of 1.

The overall efficiency of a hall thruster:
$$
\eta = \eta_m \eta_D \eta_q \eta_B \eta_V \eta_e
$$
where:
- $\eta_e$ is the electrical efficiency, i.e. how much of the total power is used for discharge (the main plasma circuit, an electric field generated between the hollow cathode and anode). It is given by, assuming no heater power since it is not given:
$$
\eta_e = \frac{V_d I_d}{V_d I_d + P_{mag}} = \frac{(300 \textrm{ V})(20 \textrm{ A})}{(300 \textrm{ V})(20 \textrm{ A}) + (31.2 \textrm{ W})}
$$
- $\eta_V$ is the voltage utilization efficiency, representing how much of the discharge voltage is effective at actually accelerating ions. It exists because there is a potential difference between the cathode and the plasma beam. The information we utilize is the given cathode voltage:
$$
\eta_V = \frac{V_d - V_c}{V_d}
$$
- $\eta_B$ is the beam utilization, representing the fraction of the supplied current from the anode that converts into accelerated ions:
$$
\eta_B = \frac{I_b}{I_d}
$$
- $\eta_q$ is the charge utilization, which accounts for the fact that the ion beam is at multiple charge states.
- $\eta_D$ is the divergence efficiency, representing losses due to ions being expelled with off-axial momentum:
$$
\eta_D  = (\cos{\theta})^2
$$
- $\eta_m$ is the mass utilization, representing how much of the propellant (neutral gas) is converted to ions, useful for thrust. The derivation involves control volume analysis but we get:
$$
\eta_m \approx \left( \frac{I_b}{q} \right) \frac{\sigma_{cs}}{u_n} \left( \frac{L_{ch}}{A_{ch}} \right)^\Gamma
$$
where $\sigma_{cs}$ is the ionization cross section, $u_n$ is the neutral velocity, $L_{ch}$ is the length of the channel, $A_{ch}$ is the cross sectional area of the channel, and $\Gamma \approx \theta \sqrt{\frac{m_i}{m_e} 0.05}$. In our case, since we do not know the geometry of the thruster, we can instead compare the mass flow rate of ions to neutrals. We know the beam current $I_b$ is carried by ions flowing out of the thruster, and by definition:
$$
I \equiv \frac{dQ}{dt}
$$
which is charge per second. So if $N$ is the number of ions per second, the current is $I_b = Ne$ where $e$ is the elementary charge. Thus to get $\dot{m}_{ion}$, we just need to multiply by the molar mass of an ion $M_{ion}$:
$$
\dot{m}_{ion} = \frac{I_b}{q} M_{ion}
$$
The molar mass of a Xenon ion is its atomic mass minus a single electon mass. So:
$$
\eta_m = \frac{\dot{m}_{ion}}{\dot{m}}
$$

Putting it all together, **we get overall $\eta = 63\%$**. The calculation done in Python is shown below.

In [81]:
import numpy as np

V_d = 300               # Discharge voltage [V]
I_d = 20                # Discharge current [A]
m_dot = 21.0402e-6      # Mass flow rate [kg/s] (converted from mg/s)
V_c = 20                # Cathode voltage [V]
P_mag = 31.2            # Magnet power [W]
I_b = 15                # Beam current [A]
theta = np.radians(15)  # Divergence half-angle [rad]
eta_q = 1.0             # Charge utilization efficiency [-]

q = 1.602e-19           # Elementary charge [C]
m_xe = 131.293          # Xenon atomic mass [amu]
m_e = 0.000548          # Electron mass [amu]
amu = 1.66054e-27       # Atomic mass unit [kg]
m_i = (m_xe-m_e) * amu  # Xenon ion mass [kg]
g_0 = 9.81              # Standard gravity [m/s^2]

eta_e = (V_d * I_d) / (V_d * I_d + P_mag)
print(f"Efficiency, electrical: {eta_e:.3f}")
eta_v = (V_d - V_c) / V_d
print(f"Efficiency, voltage: {eta_v:.3f}")
eta_b = I_b / I_d
print(f"Efficiency, beam: {eta_b:.3f}")
print(f"Efficiency, charge utilization: {eta_q:.3f}")
eta_d = np.cos(theta)**2
print(f"Efficiency, divergence: {eta_d:.3f}")
mdot_ion = ((I_b/q) * (m_xe - m_e) * amu)
eta_m = mdot_ion / m_dot
print(f"Efficiency, mass utilization: {eta_m:.3f}")
eta_total = eta_e * eta_v * eta_b * eta_q * eta_d * eta_m
print(f"Efficiency, total: {eta_total * 100:.1f}%")


Efficiency, electrical: 0.995
Efficiency, voltage: 0.933
Efficiency, beam: 0.750
Efficiency, charge utilization: 1.000
Efficiency, divergence: 0.933
Efficiency, mass utilization: 0.970
Efficiency, total: 63.0%


### Part (b)

The thrust generated from a Hall thruster can be derived by the kinetic energy of the ions exiting the thruster, which is related to the beam potential:
$$
\frac{1}{2}m_i u_e^2 = Z (V_d - V_c)
$$
Let $Z$ (the charge state) equal to $q$ since we are assuming 100% charge utilization. This is also assuming that $\langle V_B \rangle = V_d - V_c$, i.e. all ions form before being accelerated completely by the beam potential. Solving for $u_e$ and plugging into the thrust equation accounting for efficiency losses:
$$
T = \dot{m} I_{sp} g_0 = \dot{m}_{ion} \cos{\theta_D} \eta_m \eta_q \sqrt{\frac{2 q (V_d - V_c)}{m_i}}
$$

In [82]:
thrust = mdot_ion * np.cos(theta) * eta_m * eta_q * np.sqrt(2 * q * (V_d - V_c) / m_i)
thrust = np.sqrt(2 * eta_total * m_dot * (V_d * I_d + P_mag))
print(f"Thrust: {thrust:.3f} N")

Thrust: 0.400 N


### Part (c)

We're asked to account for the fact that test chambers on the ground that are used to characterize EP systems cannot pull a true vacuum, and thus overinflates the mass utilization efficiency when ambient gas ingresses into the ionization section of a thruster. We start by finding the mass flow rate of the ambient gas into the chamber using the formula provided:
$$
\dot{m}_{amb} = \frac{1}{4} m_n n_n v_n A
$$
where $m_n n_n$ is just the gas density, $v_n$ can be approximated to be the thermal velocity of the neutral gas, and $A$ is a known cross sectional area of the channel. Since the ambient gas is at very low pressure and reasonable temperatures, a callorically perfect gas (CPG) assumption is valid here, so we will use a CPG equation of state to find the ambient density:
$$
m_n n_n = \rho = \frac{p}{RT}
$$
The thermal velocity (mean of velocity magnitude in 3D) is:
$$
v_n = \sqrt{\frac{8 k_B T}{\pi m}}
$$
Now that we have $\dot{m}_{amb}$, we can adjust the mass utilization efficiency calculation. What we calculated before for mass utilization is derived from the beam current, which we can assume was backed out from an erosion rate measured with a Faraday cup or similar ion-catching device downstream of the Hall thruster during ground tests. This means the $I_b$ given in the table is the "apparent" $I_b$, and so the $\dot{m}_{ion}$ calculated from this in actuality also includes $\dot{m}_{amb}$. Thus we can find the actual $\dot{m}_{ion}$ on the ground using $I_b$ and $\dot{m}_{amb}$:
$$
\dot{m}_{ion} + \dot{m}_{amb} = \frac{I_b}{q} M_{ion}
$$
So, to calculate the thrust in space, we just use a new $\eta_m$ using only this $\dot{m}_{ion}$ we found above.

In [83]:
R_u = 8.314
k_b = 1.38e-23

torr_to_pa = 133.322
p_a = 1e-5 * torr_to_pa
T_a = 300
A = 0.02  # Channel area [m]
rho_a = p_a/((R_u/(m_xe/1e3)) * T_a)
v_n = np.sqrt((8 * k_b * T_a) / (np.pi * m_xe * amu))
mdot_a = 1/4 * rho_a * v_n * A
print(f"Ambient mass flow rate: {mdot_a*1e6:.3f} mg/s")
mdot_ion_true = ((I_b/q) * (m_xe - m_e) * amu) - mdot_a
eta_m_space = mdot_ion_true / m_dot
print(f"Mass utilization efficiency on ground: {eta_m:.3f}")
print(f"Mass utilization efficiency in space: {eta_m_space:.3f}")
T = mdot_ion_true * np.cos(theta) * eta_m_space * eta_q * np.sqrt(2 * q * (V_d - V_c) / m_i)

# Get new total efficiency in space
eta_total_space = eta_e * eta_v * eta_b * eta_q * eta_d * eta_m_space
T = np.sqrt(2 * eta_total_space * mdot_ion_true * (V_d * I_d + P_mag))
print(f"Thrust in space: {T:.3f} N")


Ambient mass flow rate: 0.077 mg/s
Mass utilization efficiency on ground: 0.970
Mass utilization efficiency in space: 0.967
Thrust in space: 0.392 N


**This gives a difference of around $8 \textrm{ mN}$.** Note that this assumes the amount of gas actually ionized is the same proportionally to both mass flow rates, and that $\eta_b$ is constant since both $I_b$ and $I_d$ are affected proportionally with respect to the changing mass flow rate.

### Part (d)

If we operate at the same discharge current in space (instead of the same internal Xenon mass flow rate), it means the efficiency that changes in space will be the $\eta_b$ instead of the $\eta_m$. This means we will proceed by carrying over the newly calculated (true) $\dot{m}_{ion}$ and converting it to an effective $I_b$ that would be in space, and deduct from thrust using the new $\eta_b$. To account for the beam utilization efficiency in thrust, we will use the classical total efficiency relation:
$$
\eta = \frac{1}{2} \left( \frac{T^2}{\dot{m}} \right) \frac{1}{P_{in}}
$$
$$
T = \sqrt{2 \eta \dot{m} P_{in}}
$$

In [84]:
I_b_true = q * mdot_ion_true/((m_xe - m_e) * amu)
eta_b_space = I_b_true / I_d
print(f"Beam efficiency in space: {eta_b_space:.3f}")
print(f"Beam efficiency on ground: {eta_b:.3f}")
eta_space = eta_e * eta_v * eta_b_space * eta_q * eta_d * eta_m
T = np.sqrt(2 * eta_space * mdot_ion_true * (V_d * I_d + P_mag))
print(f"Thrust in space: {T:.3f} N")

Beam efficiency in space: 0.747
Beam efficiency on ground: 0.750
Thrust in space: 0.392 N


As expected, the thrust calculated here is the same as the one found in Part (c).

## Problem 2: Hall Thruster Design
Assumptions:
- Propellant gas is xenon
- Optimal ion current density in the channel is $j = 100 \frac{\mathrm{mA}}{\mathrm{cm}^{2}}$
- Ratio of channel width to mean thruster diameter is constant $b/d = 0.2$
- Ratio of ionization mean free path to channel length is constant $\lambda_{\mathrm{iz}}/L = c_{\mathrm{iz}}$
- Ratio of electron Larmor radius to channel width is constant $r_{ce}/b = c_{B}$
- Ionization cross section $\sigma_{\mathrm{iz}}$ is constant
- Ions are singly charged and thruster efficiency is $1$
- Electron temperature is uniform and scales with discharge voltage $T_e = 0.1 V_0$
- Neutrals have inlet temperature $T = 800,\mathrm{K}$ and move with the thermal speed

### Part (a)
#### (i)
Given that we have 100% ionization efficiency, the ion current density must be equal to the neutral flux:
$$
j = q n_n v_n
$$
Solving for $n_n$:

In [110]:
from pint import UnitRegistry
ureg = UnitRegistry(auto_reduce_dimensions=True)
Q_ = ureg.Quantity

j = Q_(100, 'mA')/Q_(1, 'cm^2')
q = Q_(1.6022e-19, 'C')
k_b = Q_(1.3807e-23, 'J/K')
m_xe = Q_(131.293, 'amu')
T_n = Q_(800, 'K')
g_0 = Q_(9.81, 'm/s^2')

v_n = np.sqrt((8 * k_b * T_n) / (np.pi * m_xe))
n_n = j / (q * v_n)
print(f"Neutral density: {n_n.m_as('1/m^3'):.3e} 1/m^3")

Neutral density: 1.738e+19 1/m^3


#### (ii)
The density of ambient air is $1.225 \textrm{ kg/m}^3$, and we can convert this to a molar density by getting the average molar mass of air assuming 21% oxygen and 79% nitrogen by mass.

In [111]:
rho_air = Q_(1.225, 'kg/m^3')
o2_mass_fraction = 0.21
n2_mass_fraction = 0.79
M_o2 = Q_(32.0, 'g/mol')
M_n2 = Q_(28.0, 'g/mol')
# Calculate average molar mass of air
M_avg = (o2_mass_fraction * M_o2 + n2_mass_fraction * M_n2)
molar_density = rho_air / M_avg
avogadro = Q_(6.022e23, '1/mol')
n_n_air = molar_density * avogadro
print(f"Molar density of air: {n_n_air.m_as('1/m^3'):.2e} 1/m^3")
print(f"Ratio of number densities: {(n_n/n_n_air).m_as(''):.2e}")

Molar density of air: 2.56e+25 1/m^3
Ratio of number densities: 6.79e-07


#### (iii)
There is an optimal neutral density because there is an optimal ion density (here we have a 1:1 ratio because of 100% ionization). There is an optimal ion density because high density reduces the mean free path length and therefore results in more ionization, but too high of a density and there are chances for interactions between ions and between ions and neutrals; essentially it is the same as an equilibrium combustion assumption in chemical systems, where the ions are formed quickly and travel through the channel for the rest of the time. In this case, it may result in energy losses with additional interactions and collisions.

### Part (b)

Given a specific impulse requirement of 2000 seconds and various thrust requirements, finding the mean diameter $d$ involves the following procedure:
1. Finding the required neutral mass flow rate through the following equation. Because of 100% ionization, the ion mass flow rate is equal to the neutral mass flow rate:
$$
\dot{m} = \frac{F}{I_{sp} g_0}
$$
2. The area of the channel required to support this mass flow rate using the continuity equation:
$$
\dot{m} = m_n n_n v_n A_{channel}
$$
3. Area of the channel is related to the mean diameter through $A_{channel} = \pi bd$. We know $b = 0.2 d$ from the problem statement.

In [112]:
isp_req = Q_(2000, 's')
thrusts = [Q_(30, 'mN'), Q_(300, 'mN'), Q_(1, 'N')]
for t in thrusts:
    mdot_req = t / (isp_req * g_0)
    A_channel = mdot_req / (m_xe * n_n * v_n)
    d = np.sqrt(A_channel / (np.pi * 0.2))
    print(f"For thrust {t.m_as('mN'):.1f} mN, channel mean diameter: {d.m_as('cm'):.2f} cm")

For thrust 30.0 mN, channel mean diameter: 4.23 cm
For thrust 300.0 mN, channel mean diameter: 13.37 cm
For thrust 1000.0 mN, channel mean diameter: 24.42 cm


### Part (c)

Given the ionization cross section $\sigma_iz$, we can find the channel length $L$ assuming a uniform distribution of ionization probability axially.
$$
\lambda_{mfp} = \frac{1}{n_n \sigma_{iz}}
$$
The key insight is that because a neutral atom has an equal chance of ionizing along L, we know that $c_{iz} = 1/2$ since the mean length a neutral travels will be halfway through the channel given a uniform axial distribution. Rearranging:
$$
\frac{\lambda_{mfp}}{L} = \frac{1}{n_n \sigma_{iz} L} = c_{iz} = \frac{1}{2}
$$
$$
L = \frac{2}{n_n \sigma_{iz}}
$$

In [113]:
sigma_iz = Q_(5e-20, 'm^2')
L = 2/(n_n * sigma_iz)
print(f"Channel length: {L.m_as('cm'):.2f} cm")

Channel length: 230.20 cm


### Part (d)
Repeating the above but with Krypton:

In [114]:
m_kr = Q_(83.798, 'amu')
v_n_kr = np.sqrt((8 * k_b * T_n) / (np.pi * m_kr))
n_n_kr = j / (q * v_n_kr)
print(f"Krypton neutral density: {n_n_kr.m_as('1/m^3'):.3e} 1/m^3")
L = 2/(n_n_kr * sigma_iz)
print(f"Krypton channel length: {L.m_as('cm'):.2f} cm")


Krypton neutral density: 1.388e+19 1/m^3
Krypton channel length: 288.14 cm


The channel length is longer simply because the neutral density to smaller, meaning that we need a longer channel length (assuming the same distribution and still requiring 100% ionization) to achieve the same ionization.

### Part (e)

We know the magnetic field strength at a given specific impulse from the problem statement, and assuming a constant thrust, we want to find the magnetic field strength $B$ at different $I_{sp}$.

First, we'll use the constant Larmor radius to channel width from the problem statement:
$$
\frac{r_{ce}}{b} = c_B
$$
The equation for electron Larmor radius:
$$
r_{ce} = \frac{m_e v_e}{q B}
$$
where $v_e$ is the thermal velocity of an electron, which depends on electron temperature:
$$
v_e = \sqrt{\frac{k_B T_e}{m_e}}
$$
and since we know from the problem statement that $T_e = 0.1 V_0$, we know that as a scaling law (ignoring constants):
$$
v_e \propto \sqrt{V_0}
$$
and so:
$$
r_{ce} \propto \frac{\sqrt{V_0}}{B}
$$

Next, we can look at relating thrust to the discharge voltage. From previous derivation, we know we can relate $\dot{m}_{ion}$ to beam current through:
$$
\dot{m}_{ion} = \frac{I_b m_i}{e}
$$
and that exhaust velocity can be derived from kinetic energy and related to $V_0$ (assuming 100% of discharge potential is used):
$$
\frac{1}{2} m_i u_e^2 = q V_0
$$
Putting the previous two equations together through the thrust equation, we get:
$$
T = I_b \sqrt{\frac{2 m_i V_0}{q}}
$$

Since from the problem statement we can assume **thrust is constant**, we can rearrange this to get a scaling law for $I_b$:
$$
I_b \propto \frac{1}{\sqrt{V_0}} \propto V_0^{-1/2}
$$
And we saw before that by definition of $j$ (which is also constant):
$$
I_b = jA \propto d^2
$$
And so we can combine the two equations to relate $b$ (since it is directly proportional to $b$ by a constant of 0.2) to $V_0$:
$$
d^2 \propto I_b \propto V_0^{-1/2}
$$
$$
b \propto V_0^{-1/4}
$$

Finally, putting everything together:
$$
r_{ce} \propto \frac{\sqrt{V_0}}{B}
$$
$$
b \propto V_0^{-1/4}
$$
And since $r_{ce}/b$ is constant:
$$
\frac{r_{ce}}{b} \propto \frac{\frac{\sqrt{V_0}}{B}}{V_0^{-1/4}} \propto \frac{V_0^{3/4}}{B} = \textrm{ constant}
$$
$$
\boxed{B \propto V_0^{3/4}}
$$
Plugging in $I_{sp}$ from the $u_e = I_{sp}g_0$ and the relation of kinetic energy to discharge voltage, we get:
$$
B \propto I_{sp}^{3/2}
$$
And so we can relate magnetic field strengths to specific impulse through:
$$
\boxed{\frac{B_2}{B_1} = \left( \frac{I_{sp,2}}{I_{sp,1}} \right)^{3/2}}
$$

In [116]:
isp_1 = 2000
B_1 = 200
isp_i = 1500
isp_ii = 3000
B_i = B_1 * (isp_i/isp_1)**(3/2)
B_ii = B_1 * (isp_ii/isp_1)**(3/2)
print(f"Magnetic field for Isp = {isp_i} s: {B_i:.1f} G")
print(f"Magnetic field for Isp = {isp_ii} s: {B_ii:.1f} G")

Magnetic field for Isp = 1500 s: 129.9 G
Magnetic field for Isp = 3000 s: 367.4 G
